In [3]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import itertools
from rosemary import jpt_in_notebook
from llm.submit import submit_job, multiline_to_singleline, shell_scripts_template_slurm


In [7]:
from note_pruning_analysis import get_dataset

# ds = get_dataset('flan2022/niv2_fs_noopt_train.jsonl', processed=False)

In [8]:
ds1 = get_dataset('flan_v2/flan_v2_resampled_100k.jsonl', processed=False)
ds2 = get_dataset('flan2022/flan2022_1m.jsonl', processed=False)
ds3 = get_dataset('flan2022/flan2022v1_1m.jsonl', processed=False)

Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/raw_train/json/default-1d7a878687ccf868/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/raw_train/json/default-e63bbb75ae936d4f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/raw_train/json/default-5ebf04b734e102f1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [ ]:
from collections import Counter

ds = ds1
for i, ds in enumerate([ds1, ds2]):
    print(i)
    for k in ['_task_source', '_template_type']:
        counts = dict(Counter(ds[k]))
        s = sum(counts.values())
        counts = {k: v/s for k, v in counts.items()}
        print(k, counts)
    break

In [ ]:
for i in range(200, 300):
    print(i)
    s = ds1[i]['inputs'][:100]
    dsf = ds2.filter(lambda x: x['inputs'].startswith(s), num_proc=32, keep_in_memory=True)
    if len(dsf)!=0:
        s = ds1[i]['targets'][:100]
        dsf = dsf.filter(lambda x: x['targets'].startswith(s), num_proc=4, keep_in_memory=True)
        if len(dsf)!=0:
            break
    

In [15]:
test_run = 0
test_run = bool(test_run)

portions_list = ['flan2022v1', 'flan2022v2']

N = 1000000; N_str = '1m'
N =  100000; N_str = '100k'
# N =   50000; N_str = '50k'

save_dir = 'data/raw_train/flan2022/'

for portions in portions_list:
    output_filename = f'{portions}_{N_str}.jsonl'
    
    cmd = f"""
    python resample_flan_v2.py \
        --flan_v2_data_dir data/raw_train/flan2022 \
        --total_num_samples {N} \
        --output_path {os.path.join(save_dir, output_filename)} \
        --portions_type {portions} \
    """
    cmd = multiline_to_singleline(cmd)
    shell_scripts = shell_scripts_template_slurm.format(
        conda_env='open-instruct',
        cwd=os.getcwd(),
        cmd=cmd,
        log_dir=os.getcwd(),
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=f'resample_flan_v2', 
        nodes=1,
        num_cpus=64,
        cpu_mem=128,
        num_gpus=1,
        gpu_type='v100',
        test_run=test_run,
        job_duration=6,
    )
    print(cmd)


Submiting job with:
{
    "job_name": "resample_flan_v2",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python resample_flan_v2.py --flan_v2_data_dir data/raw_train/flan2022 --total_num_samples 100000 --output_path data/raw_train/flan2022/flan2022v1_100k.jsonl --portions_type flan2022v1

Submiting job with:
{
    "job_name": "resample_flan_v2",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python resample_flan_v2.py --flan_v2_data_dir data/raw_train/flan2022 --total_num_samples 100000 --output_path data/raw_train/flan2022/flan2022v2_100k.jsonl --portions_type flan2022v2


In [ ]:

# python open_instruct/reformat_datasets.py --raw_data_dir data/raw_train --output_dir data/processed --dataset flan2022



